# Chapter 5: The Inner Sea in the Tragedies of Harpur and Baillie

Charles Harpur and Joanna Baillie were two of the preeminent tragideans of the Romantic period. Baillie achieved enormous fame in her lifetime, and considerable success on the stage. Harpur achieved considerable fame in New South Wales, where he was born and worked, and has posthumously been acclaimed by nationalist critics as the founder of Australia's dramatic tradition. In their theory and practice, these two dramatists pursued different tracks. While both were interested in forms of subjective drama, which portrayed the destruction or breakdown of the human psyche, Baillie's drama was a drama of interiority and psychology, while Harpur's drama was a drama of outward action and sociality. This notebook helps to draw this contrast between Baillie and Harpur, by using network analysis and text analysis to reveal the different structures of their plays.

The analysis in the notebook corresponds to Section 5.2 of *Romanticism and the Contingent Self.*

In [1]:
from math import inf

from romanticself import import_network_data, view_characters, view_community_keywords
import pydracor as dc
import pandas as pd
import numpy as np

corpus = import_network_data("data/drama-networks/", use_cache=False)

/opt/homebrew/Caskroom/miniforge/base/envs/py11/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [2]:
corpus["harpur"].es

In [3]:
view_characters(corpus, "harpur", "community", n=10)

Showing community for top 10 characters in The Bushrangers by word count

Stalwart                 : 2
Ada                      : 3
Bomebard                 : 1
Tunbelly                 : 0
Abel                     : 3
Macblood                 : 2
Dreadnought              : 1
Lucy Grey                : 3
Rackroad                 : 2
Filch                    : 2


In [4]:
view_community_keywords(corpus, "harpur")

Showing most distinctive words by community in The Bushrangers

0        : you, respect, ned, the, and, of, a, to, have, i, your, sir, is, but, in, bomebard, worship...
1        : the, neighbour, you, bomebard, i, says, and, a, o, to, worships, ned, but, my, me, does, s...
2        : ranger, free, welcome, forest, died, strife, the, lived, and, a, you, thee, to, we, me, tr...
3        : ada, you, the, abel, lucy, and, i, of, but, to, she, a, her, me, through, in, sun, not, th...
4        : hoo, he, farmer, needn, didn, t, stalwart, swear, shot, doesn, after, worship, crabtree, s...
5        : guns, a, i, the, and, of, to, fie, gun, walking-stick, further, invented, lot, a-running, ...


## Comparing the overall structures of the plays

This first table compares the overall network topology of the seven plays in our small corpus with the network topology of a large number of plays in [DraCor](https://dracor.org).

In [5]:
dracor_df = pd.DataFrame.from_records([
    {
        "Corpus": corpus.title,
        "Year": play.get("year_normalized"),
        "Genre": play.get("normalized_genre"),
        "Size": play.get("num_of_speakers"),
        "Density": play.get("density"),
        "Clustering Coefficient": play.get("average_clustering"),
        "Average Path Length": play.get("average_path_length"),
        "ID": play.get("id")
    }
    for corpus in (dc.Corpus(lang) for lang in ["ger","fre","rus"])
    for play in corpus.metadata()
    if play.get("normalized_genre") == "Tragedy"
])

display(dracor_df)

,Corpus,Year,Genre,Size,Density,Clustering Coefficient,Average Path Length,ID
0,German Drama Corpus,1779,Tragedy,20,0.178947,0.642686,1.915584,ger000588
1,German Drama Corpus,1790,Tragedy,21,0.480952,0.825706,1.538095,ger000282
2,German Drama Corpus,1823,Tragedy,10,0.444444,0.615714,1.622222,ger000455
3,German Drama Corpus,1828,Tragedy,57,0.122807,0.818361,2.646707,ger000090
4,German Drama Corpus,1889,Tragedy,23,0.830040,0.910614,1.169960,ger000322
...,...,...,...,...,...,...,...,...
478,Russian Drama Corpus,1751,Tragedy,10,0.400000,0.685476,1.688889,rus000010
479,Russian Drama Corpus,1750,Tragedy,6,0.666667,0.805556,1.333333,rus000116
480,Russian Drama Corpus,1866,Tragedy,70,0.137474,0.819953,2.175966,rus000076
481,Russian Drama Corpus,1870,Tragedy,102,0.205591,0.959723,1.971399,rus000082


In [10]:
def period(year: int, breaks: list[int]) -> str:
    _breaks = [-inf, *breaks, inf]
    idxs = [idx for idx,val in enumerate(_breaks) if year < val]
    end = _breaks[min(idxs)]
    end = end - 1 if end != inf else ""
    start = _breaks[min(idxs) - 1]
    start = start if start != -inf else ""
    return f"{start}-{end}"

dracor_grouped = (dracor_df.assign(
    Year = [period(year, [1740, 1770, 1800, 1830]) for year in dracor_df["Year"]],
    ID = None)
    .groupby(["Corpus","Year"]))

In [11]:
dracor_analysis = dracor_grouped.mean(numeric_only=True).assign(Play = dracor_grouped.size()).reset_index()

In [13]:

contingent_self_analysis = pd.DataFrame(data= {
   "Corpus": "Contingent Self Corpus",
   "Year": [graph['date'] for graph in corpus.values()],
   "Play": [graph['title']for graph in corpus.values()],
   "Size": [len(graph.vs) for graph in corpus.values()],
   "Density": [graph.density() for graph in corpus.values()],
   "Clustering Coefficient": [graph.transitivity_avglocal_undirected(mode="zero") for graph in corpus.values()],
   "Average Path Length": [graph.average_path_length() for graph in corpus.values()],
   "Classes": [max(graph.vs["community"]) for graph in corpus.values()],
   "Modularity": [graph.modularity(graph.vs["community"], weights="weight") for graph in corpus.values()]
})

In [14]:
figure_5_2 = pd.concat([contingent_self_analysis, dracor_analysis])
figure_5_2.to_csv("figures/figure_5_2.csv")
display(figure_5_2)

,Corpus,Year,Play,Size,Density,Clustering Coefficient,Average Path Length,Classes,Modularity
0,Contingent Self Corpus,1812,The Dream,20.000000,0.313158,0.686881,1.821637,5.0,0.234918
1,Contingent Self Corpus,1853,The Bushrangers,27.000000,0.188034,0.715722,2.672308,5.0,0.465330
2,Contingent Self Corpus,1823,Vespers of Palermo,18.000000,0.307190,0.477329,2.044118,1.0,0.022368
3,Contingent Self Corpus,1674,Iphigénie,11.000000,0.427273,0.602814,1.760000,3.0,0.235613
4,Contingent Self Corpus,1812,Orra,17.000000,0.522059,0.664772,1.562500,1.0,0.197010
5,Contingent Self Corpus,1606,Macbeth,42.000000,0.109756,0.485837,2.474703,7.0,0.431431
6,Contingent Self Corpus,1813,Remorse,15.000000,0.257143,0.461270,1.884615,7.0,0.372760
0,French Drama Corpus,-1739,231,11.255411,0.494987,0.693271,1.584154,NaN,NaN
1,French Drama Corpus,1740-1769,36,9.222222,0.573652,0.709811,1.507406,NaN,NaN
2,French Drama Corpus,1770-1799,28,10.821429,0.523037,0.695482,1.498540,NaN,NaN
